In [1]:

import numpy as np
import pandas as pd
import os
import scipy
import matplotlib.pyplot as plt, seaborn as sns
import librosa
from tqdm import tqdm

import IPython.display as ipd

In [8]:
import sys
sys.version

'3.10.12 (main, Nov 20 2023, 15:14:05) [GCC 11.4.0]'

In [2]:
DATASET_AUDIO_PATH = '/content/drive/MyDrive/Praca_Inżynierska_Dominik_Ossowski/Dane_Inżynierka_Dominik_Ossowski/audio_medium (1)'

In [7]:
print(len([entry for entry in os.listdir(DATASET_AUDIO_PATH) if os.path.isfile(os.path.join(DATASET_AUDIO_PATH, entry))]))

24979


In [5]:
tracks = pd.read_csv('/content/drive/MyDrive/tracks.csv', index_col=0, low_memory=False)
tracks.head()

,album,album.1,album.2,album.3,album.4,album.5,album.6,album.7,album.8,album.9,...,track.10,track.11,track.12,track.13,track.14,track.15,track.16,track.17,track.18,track.19
NaN,comments,date_created,date_released,engineer,favorites,id,information,listens,producer,tags,...,information,interest,language_code,license,listens,lyricist,number,publisher,tags,title
track_id,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0,2008-11-26 01:44:45,2009-01-05 00:00:00,NaN,4,1,<p></p>,6073,NaN,[],...,NaN,4656,en,Attribution-NonCommercial-ShareAlike 3.0 Inter...,1293,NaN,3,NaN,[],Food
3,0,2008-11-26 01:44:45,2009-01-05 00:00:00,NaN,4,1,<p></p>,6073,NaN,[],...,NaN,1470,en,Attribution-NonCommercial-ShareAlike 3.0 Inter...,514,NaN,4,NaN,[],Electric Ave
5,0,2008-11-26 01:44:45,2009-01-05 00:00:00,NaN,4,1,<p></p>,6073,NaN,[],...,NaN,1933,en,Attribution-NonCommercial-ShareAlike 3.0 Inter...,1151,NaN,6,NaN,[],This World


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Z zagnieżdżonego dataframeu wyodrębniłem potrzebne kolumny - gatunek top, gatunki oraz do jakiego podzbioru należą

In [6]:
track_info = tracks[["track.7",'track.8',"set.1"]] #prerobić dla set.1 medium
track_info.columns = track_info.iloc[0].rename('track_id')
track_info = track_info.iloc[2:]

In [7]:
track_info

track_id,genre_top,genres,subset
2,Hip-Hop,[21],small
3,Hip-Hop,[21],medium
5,Hip-Hop,[21],small
10,Pop,[10],small
20,NaN,"[76, 103]",large
...,...,...,...
155316,Rock,[25],large
155317,Rock,[25],large
155318,Rock,[25],large
155319,Rock,[25],large


In [8]:
track_info.loc[pd.isnull(track_info).any(1)]

<ipython-input-8-d3096f3b0494>:1: FutureWarning: In a future version of pandas all arguments of DataFrame.any and Series.any will be keyword-only.
  track_info.loc[pd.isnull(track_info).any(1)]


track_id,genre_top,genres,subset
20,NaN,"[76, 103]",large
26,NaN,"[76, 103]",large
30,NaN,"[76, 103]",large
46,NaN,"[76, 103]",large
48,NaN,"[76, 103]",large
...,...,...,...
155276,NaN,"[15, 32, 38]",large
155277,NaN,"[15, 32, 38]",large
155278,NaN,"[42, 107]",large
155288,NaN,[],large


In [9]:
track_info.genre_top.value_counts()

Rock                   14182
Experimental           10608
Electronic              9372
Hip-Hop                 3552
Folk                    2803
Pop                     2332
Instrumental            2079
International           1389
Classical               1230
Jazz                     571
Old-Time / Historic      554
Spoken                   423
Country                  194
Soul-RnB                 175
Blues                    110
Easy Listening            24
Name: genre_top, dtype: int64

In [10]:
track_info.genre_top.value_counts().sum()

49598

W kolumnie gatunki_top pojawiły się brakujące wartości. Dzieje się tak, dlatego że dany utwór można zaliczyć do więcej niż jednego gatunku, o czym informuje kolumna gatunki.

In [11]:
track_info_wo_genre = track_info.loc[pd.isnull(track_info).any(1)]
genres_df = pd.read_csv('/content/drive/MyDrive/genres.csv')
genre_names = []
for i in track_info_wo_genre.genres:
    j = eval(i)+[0]
    if j[0] != 0:
        top_level = genres_df[genres_df.genre_id==j[0]].top_level.values[0]
        parent_genre = genres_df[genres_df.genre_id==top_level]['title'].values[0]
    else:
        parent_genre = np.nan
    genre_names.append(parent_genre)
track_info_ = track_info.copy()
track_info_.loc[track_info_wo_genre.index, 'genre_top'] = genre_names

<ipython-input-11-65522c24e24f>:1: FutureWarning: In a future version of pandas all arguments of DataFrame.any and Series.any will be keyword-only.
  track_info_wo_genre = track_info.loc[pd.isnull(track_info).any(1)]


Zastąpiłem brakujące wartości w kolumnie gatunki_top pierwszym gatunkiem z list w kolumnie gatunki.

In [12]:
track_info_.loc[pd.isnull(track_info_).any(1)]

<ipython-input-12-dc921d542ced>:1: FutureWarning: In a future version of pandas all arguments of DataFrame.any and Series.any will be keyword-only.
  track_info_.loc[pd.isnull(track_info_).any(1)]


track_id,genre_top,genres,subset
613,NaN,[],large
1213,NaN,[],large
1216,NaN,[],large
1217,NaN,[],large
3276,NaN,[],large
...,...,...,...
155192,NaN,[],large
155193,NaN,[],large
155194,NaN,[],large
155195,NaN,[],large


Nadal znajdują się wartości puste, dlatego trzeba je usunąć

In [13]:
track_info_.genre_top.value_counts()

Electronic             25046
Experimental           22724
Rock                   21282
Pop                     8015
Folk                    6083
Hip-Hop                 4534
Instrumental            4367
International           2924
Jazz                    2293
Classical               2102
Blues                   1676
Soul-RnB                 920
Spoken                   768
Country                  726
Old-Time / Historic      710
Easy Listening           173
Name: genre_top, dtype: int64

In [14]:
track_info_.genre_top.value_counts().sum()

104343

In [15]:
track_info_.drop(track_info_[track_info_['subset'] == "large"].index, inplace = True)

Z racji, że korzystam z podzbioru medium, usuwam wartości otagowane jako large

In [16]:
track_info_.genre_top.value_counts()

Rock                   7103
Electronic             6314
Experimental           2251
Hip-Hop                2201
Folk                   1519
Instrumental           1350
Pop                    1186
International          1018
Classical               619
Old-Time / Historic     510
Jazz                    384
Country                 178
Soul-RnB                154
Spoken                  118
Blues                    74
Easy Listening           21
Name: genre_top, dtype: int64

In [17]:
track_info_.genre_top.value_counts().sum()

25000

In [18]:
track_info_['new'] = track_info_.index
track_info_

track_id,genre_top,genres,subset,new
2,Hip-Hop,[21],small,2
3,Hip-Hop,[21],medium,3
5,Hip-Hop,[21],small,5
10,Pop,[10],small,10
134,Hip-Hop,[21],medium,134
...,...,...,...,...
155297,Instrumental,"[18, 107, 1235]",medium,155297
155298,Folk,"[17, 103]",medium,155298
155306,Folk,"[17, 103]",medium,155306
155307,Experimental,[1],medium,155307


In [19]:
track_info_.drop(track_info_[track_info_["new"] == "98571"].index,inplace = True)
track_info_.drop(track_info_[track_info_["new"] == "98559"].index,inplace = True)
track_info_.drop(track_info_[track_info_["new"] == "5574"].index,inplace = True)
track_info_.drop(track_info_[track_info_["new"] == "126981"].index,inplace = True)
track_info_.drop(track_info_[track_info_["new"] == "99134"].index,inplace = True)
track_info_.drop(track_info_[track_info_["new"] == "105247"].index,inplace = True)
track_info_.drop(track_info_[track_info_["new"] == "108925"].index,inplace = True)
track_info_.drop(track_info_[track_info_["new"] == "98558"].index,inplace = True)
track_info_.drop(track_info_[track_info_["new"] == "98560"].index,inplace = True)
track_info_.drop(track_info_[track_info_["new"] == "80391"].index,inplace = True)
track_info_.drop(track_info_[track_info_["new"] == "127336"].index,inplace = True)
track_info_.drop(track_info_[track_info_["new"] == "65753"].index,inplace = True)
track_info_.drop(track_info_[track_info_["new"] == "143992"].index,inplace = True)
track_info_.drop(track_info_[track_info_["new"] == "133297"].index,inplace = True)
track_info_.drop(track_info_[track_info_["new"] == "1486"].index,inplace = True)
track_info_.drop(track_info_[track_info_["new"] == "98567"].index,inplace = True)
track_info_.drop(track_info_[track_info_["new"] == "98569"].index,inplace = True)
track_info_.drop(track_info_[track_info_["new"] == "98568"].index,inplace = True)
track_info_.drop(track_info_[track_info_["new"] == "98566"].index,inplace = True)
track_info_.drop(track_info_[track_info_["new"] == "98565"].index,inplace = True)
track_info_.drop(track_info_[track_info_["new"] == "108924"].index,inplace = True)

Powyżej znajdują się utwory, których długość wynosiła mniej niż 29 sekund albo były uszkodzone

In [20]:
track_info_

track_id,genre_top,genres,subset,new
2,Hip-Hop,[21],small,2
3,Hip-Hop,[21],medium,3
5,Hip-Hop,[21],small,5
10,Pop,[10],small,10
134,Hip-Hop,[21],medium,134
...,...,...,...,...
155297,Instrumental,"[18, 107, 1235]",medium,155297
155298,Folk,"[17, 103]",medium,155298
155306,Folk,"[17, 103]",medium,155306
155307,Experimental,[1],medium,155307


In [21]:
track_info_.genre_top.value_counts()

Rock                   7097
Electronic             6311
Experimental           2250
Hip-Hop                2192
Folk                   1518
Instrumental           1349
Pop                    1186
International          1018
Classical               619
Old-Time / Historic     510
Jazz                    384
Country                 178
Soul-RnB                154
Spoken                  118
Blues                    74
Easy Listening           21
Name: genre_top, dtype: int64

In [ ]:
track_info_['audio_name'] = os.listdir(DATASET_AUDIO_PATH)

In [27]:
track_info_.drop(track_info_[track_info_['genre_top'] == "Easy Listening"].index, inplace = True)
track_info_.drop(track_info_[track_info_['genre_top'] == "Blues"].index, inplace = True)

In [28]:
track_info_clean = track_info_.drop(columns=['genres', 'subset', 'new'])
track_info_clean.dropna()
track_info_clean

track_id,genre_top,audio_name
2,Hip-Hop,102168.mp3
3,Hip-Hop,125585.mp3
5,Hip-Hop,085488.mp3
10,Pop,097204.mp3
134,Hip-Hop,015157.mp3
...,...,...
155297,Instrumental,120465.mp3
155298,Folk,003459.mp3
155306,Folk,072608.mp3
155307,Experimental,124498.mp3


Poniżej obliczam mfcc dla każdego utworu i zapisuje go do dataframeu (zapisuje dwie wesje - mean oraz bez operacji mean)

In [29]:
extracted_features=[]
extracted_features_mean=[]

In [30]:
def features_extractor(file):
    audio, sample_rate = librosa.load(file_name, res_type='soxr_hq')
    mfccs_features = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=13, n_fft = 2048, hop_length=512)
    mfccs_scaled_features = mfccs_features.T
    mfccs_scaled_features_mean = np.mean(mfccs_features.T,axis=0)

    return mfccs_scaled_features, mfccs_scaled_features_mean

In [ ]:
for index_num,row in tqdm(track_info_clean.iterrows()): #.iloc[15628:]
    file_name = os.path.join(os.path.abspath(DATASET_AUDIO_PATH), row['audio_name'])
    final_class_labels=row["genre_top"]
    mfccs_scaled_features, mfccs_scaled_features_mean=features_extractor(file_name)
    #extracted_features.append([data,final_class_labels])
    extracted_features.append([mfccs_scaled_features, final_class_labels])
    extracted_features_mean.append([mfccs_scaled_features_mean, final_class_labels])

In [ ]:
extracted_features

[[array([[-331.41824   ,   95.99606   ,  -25.466045  , ...,  -27.784924  ,
           -14.051747  ,  -22.497324  ],
         [-202.20712   ,   94.473335  ,  -35.637062  , ...,  -27.238365  ,
            -8.318442  ,  -14.167865  ],
         [-159.35538   ,   65.31082   ,  -19.200739  , ...,  -21.001955  ,
            -5.6345043 ,  -11.893929  ],
         ...,
         [ -37.86642   ,   39.07254   ,   -0.3861336 , ...,  -11.22114   ,
            -1.1253824 ,    2.4148598 ],
         [  -5.3553667 ,   51.98365   ,  -13.983837  , ...,  -16.516981  ,
            -8.810869  ,    0.41869795],
         [  -3.1073823 ,   63.974342  ,  -27.302168  , ...,   -9.495638  ,
            -1.5654974 ,   -0.92783153]], dtype=float32),
  'Hip-Hop'],
 [array([[-308.35553  ,   69.364334 ,    5.986863 , ...,   -3.368579 ,
            14.428613 ,    2.327971 ],
         [-134.9586   ,   71.989    ,   -6.7240896, ...,   -2.2001238,
             1.0482353,  -10.58997  ],
         [ -76.38166  ,   60.877426 ,  

In [ ]:
extracted_features_v2 = pd.DataFrame(extracted_features, columns=['features','genre'])
extracted_features_v2

,features,genre
0,"[[-331.41824, 95.99606, -25.466045, 18.971329,...",Hip-Hop
1,"[[-308.35553, 69.364334, 5.986863, 47.842987, ...",Hip-Hop
2,"[[-349.44122, 127.23545, -4.3016386, 4.849605,...",Hip-Hop
3,"[[-209.57384, 44.123947, -34.430008, 13.012472...",Pop
4,"[[-282.58325, 48.749207, 78.15404, 1.7475528, ...",Hip-Hop
...,...,...
24879,"[[-447.8207, 137.37451, -8.227509, 17.44024, 1...",Instrumental
24880,"[[-190.82816, 143.20755, 10.690748, 73.98483, ...",Folk
24881,"[[-81.570755, 108.5269, 2.0659103, 19.400057, ...",Folk
24882,"[[-175.63293, 91.08409, -80.6078, 13.257477, -...",Experimental


In [ ]:
extracted_features_v1 = pd.DataFrame(extracted_features_mean, columns=['features','genre'])
extracted_features_v1

In [ ]:
extracted_features_v2.shape

(24884, 2)

In [ ]:
extracted_features_v2.to_pickle('extracted_features_soxr_hq_not_mean.df')

In [ ]:
extracted_features_v2.genre.value_counts()

Rock                   7097
Electronic             6311
Experimental           2250
Hip-Hop                2192
Folk                   1518
Instrumental           1349
Pop                    1186
International          1018
Classical               619
Old-Time / Historic     510
Jazz                    384
Country                 178
Soul-RnB                154
Spoken                  118
Name: genre, dtype: int64

In [ ]:
extracted_features_v2.genre.value_counts().sum()

24884

Przycięcie danych

Okazało się, że utwory nie miały dokłanie 30 sekund (kilkanaście milisekund więcej lub mniej), dlatego należy przyciąć klatki do równej ilości

In [ ]:
extracted_features_copy_all = pd.read_pickle("C:/Users/Dominik/Downloads/extracted_features_kaiser_fast_not_mean.df")

In [ ]:
extracted_features_copy_all

,features,genre
0,"[[-331.5843505859375, 96.2279052734375, -25.68...",Hip-Hop
1,"[[-310.6990966796875, 72.66214752197266, 2.738...",Hip-Hop
2,"[[-349.44122314453125, 127.2354965209961, -4.3...",Hip-Hop
3,"[[-216.35218811035156, 53.68293762207031, -43....",Pop
4,"[[-290.0492858886719, 59.281185150146484, 67.7...",Hip-Hop
...,...,...
24878,"[[-447.8392333984375, 137.40016174316406, -8.2...",Instrumental
24879,"[[-194.3589324951172, 148.20565795898438, 5.68...",Folk
24880,"[[-87.11096954345703, 116.3446044921875, -5.70...",Folk
24881,"[[-178.5210418701172, 95.12547302246094, -84.6...",Experimental


In [ ]:
extracted_features_copy_all.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24883 entries, 0 to 24882
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   features  24883 non-null  object
 1   genre     24883 non-null  object
dtypes: object(2)
memory usage: 388.9+ KB


In [ ]:
data = extracted_features_copy_all['features']
len(data)

24883

In [ ]:
c = 0
errors =[]
for index, row in extracted_features_copy_all.iterrows():
    if len(row['features']) == 1291:
        c = c + 1
    else:
        errors.append(row['features'])

In [ ]:
c

13356

In [ ]:
d = 0
errors_v2 = []
for index, row in extracted_features_copy_all.iterrows():
    if len(row['features']) == 1291:
        d = d + 1
    else:
        errors_v2.append(len(row['features']))

In [ ]:
error = pd.DataFrame(errors_v2, columns=['error'])

In [ ]:
error['error'].unique()

array([1293, 1292], dtype=int64)

Obcinam 2 końcowe klatki

In [ ]:
extracted_features_copy_all['modified_features'] = extracted_features_copy_all['features']
for index, row in extracted_features_copy_all.iterrows():
    if len(row['features']) == 1293:
        row['modified_features'] = row['features'][:-2]
    else:
        if len(row['features']) == 1292:
            row['modified_features'] = row['features'][:-1]

extracted_features_copy_all.drop('features', axis=1, inplace=True)

In [ ]:
extracted_features_copy_all.rename(columns={'modified_features': 'features'}, inplace=True)

In [ ]:
extracted_features_copy_all

,genre,features
0,Hip-Hop,"[[-331.5843505859375, 96.2279052734375, -25.68..."
1,Hip-Hop,"[[-310.6990966796875, 72.66214752197266, 2.738..."
2,Hip-Hop,"[[-349.44122314453125, 127.2354965209961, -4.3..."
3,Pop,"[[-216.35218811035156, 53.68293762207031, -43...."
4,Hip-Hop,"[[-290.0492858886719, 59.281185150146484, 67.7..."
...,...,...
24878,Instrumental,"[[-447.8392333984375, 137.40016174316406, -8.2..."
24879,Folk,"[[-194.3589324951172, 148.20565795898438, 5.68..."
24880,Folk,"[[-87.11096954345703, 116.3446044921875, -5.70..."
24881,Experimental,"[[-178.5210418701172, 95.12547302246094, -84.6..."


In [ ]:
d = 0
errors_v2 = []
for index, row in extracted_features_copy_all.iterrows():
    if len(row['features']) == 1291:
        d = d + 1
    else:
        errors_v2.append(len(row['features']))

In [ ]:
error = pd.DataFrame(errors_v2, columns=['error'])
error['error'].unique()

array([], dtype=object)